In [1]:
import cobra
from cobra import Model
from typing import List
from abc import ABC, abstractmethod
from diel_models.compartments_creator import CompartmentsCreator
from diel_models.storage_pool_creator import StoragePoolCreator
from diel_models.photon_reaction_restrictor import PhotonReactionRestrictor
from diel_models.biomass_adjuster import BiomassAdjuster


class Step(ABC):
    @abstractmethod
    def run(self):
        pass

    def validate(self):
        pass


class Compartments(Step):

    def __init__(self, model: Model):
        self.model = model

    def run(self):
        test = CompartmentsCreator(self.model)
        test.day_attribution()
        test.duplicate()

    def validate(self):
        assert len(self.model.reactions) > 0, "The model does not have any reaction."
        assert len(self.model.metabolites) > 0, "The model does not have any metabolite."
        assert self.model.objective is not None, "The model does not have any objective function defined."


class StoragePool(Step):

    def __init__(self, model: Model, metabolites: List[str]):
        self.model = model
        self.metabolites = metabolites

    def run(self):
        test = StoragePoolCreator(self.model, self.metabolites)
        test.create_storage_pool_metabolites()
        test.create_storage_pool_first_reactions()
        test.create_storage_pool_second_reactions()

    def validate(self):

        for reaction in self.model.reactions:
            assert "_Day" in reaction.id or "_Night" in reaction.id, "The model does not have Day and Night reactions."
        for metabolite in self.model.metabolites:
            assert "_Day" in metabolite.id or "_Night" in metabolite.id, "The model does not have Day and Night " \
                                                                         "metabolites."
        for compartment in self.model.compartments:
            assert "_Day" in compartment or "_Night" in compartment, "The model does not have Day and Night " \
                                                                     "compartments."


class PhotonRestrictor(Step):

    def __init__(self, model: Model, id_photon_reaction_night: str):
        self.model: Model = model
        self.id_photon_reaction_night: str = id_photon_reaction_night

    def run(self):
        test = PhotonReactionRestrictor(self.model, self.id_photon_reaction_night)
        test.restrain()

    def validate(self):

        for compartment in self.model.compartments:
            assert "_Day" in compartment or "_Night" in compartment or 'sp' in compartment, "The model does not have " \
                                                                                            "storage pool compartment"

        for metabolite in self.model.metabolites:
            if metabolite.compartment == "sp":
                assert "_sp" in metabolite.id, "The storage pool metabolites does not have sp suffix in their id"
                assert "Day" not in metabolite.name or "Night" not in metabolite.name, "The storage pool metabolites " \
                                                                                       "continue to have Day or Night" \
                                                                                       " in their name"


class Biomass(Step):

    def __init__(self, model: Model, id_biomass_reaction_day: str, id_biomass_reaction_night: str,
                 photosynthesis_reactions_at_night: List[str]):
        self.model: Model = model
        self.id_biomass_reaction_day: str = id_biomass_reaction_day
        self.id_biomass_reaction_night: str = id_biomass_reaction_night
        self.photosynthesis_reactions_at_night: List[str] = photosynthesis_reactions_at_night

    def run(self):
        test = BiomassAdjuster(self.model, self.id_biomass_reaction_day, self.id_biomass_reaction_night,
                               self.photosynthesis_reactions_at_night)
        test.reset_boundaries()
        test.total_biomass_reaction()

    def validate(self):
        pass


class Pipeline:
    def __init__(self, model: Model, steps):
        self.steps = steps
        self.model = model

    def run(self):
        for step in self.steps:
            step.validate()
            step.run()
            cobra.io.write_sbml_model(self.model, "Model.xml")
            self.model = cobra.io.read_sbml_model("Model.xml")

In [2]:
import os
from tests import TEST_DIR

aragem_model_path = os.path.join(TEST_DIR, 'data', 'aragem_photo.xml')

modelo = cobra.io.read_sbml_model(aragem_model_path)

Steps = [
    Compartments(modelo),
    StoragePool(modelo, ["S_Sucrose_c[C_c]_Day", "S_Sulfate_c[C_c]_Day", "S_Nitrate_c[C_c]_Day", "S_L_45_Histidine_c[C_c]_Day", "S_L_45_Isoleucine_c[C_c]_Day", "S_L_45_Leucine_c[C_c]_Day", "S_L_45_Lysine_c[C_c]_Day", "S_L_45_Methionine_c[C_c]_Day", "S_L_45_Phenylalanine_c[C_c]_Day", "S_L_45_Threonine_c[C_c]_Day", "S_L_45_Tryptophan_c[C_c]_Day", "S_L_45_Valine_c[C_c]_Day", "S_L_45_Arginine_c[C_c]_Day", "S_L_45_Cysteine_c[C_c]_Day", "S_L_45_Glutamine_c[C_c]_Day", "S_L_45_Glutamate_c[C_c]_Day", "S_Glycine_c[C_c]_Day", "S_L_45_Proline_c[C_c]_Day", "S_L_45_Tyrosine_c[C_c]_Day", "S_L_45_Alanine_c[C_c]_Day", "S_L_45_Asparagine_c[C_c]_Day", "S_L_45_Serine_c[C_c]_Day", "S_Orthophosphate_c[C_c]_Day", "S_Starch_p[C_p]_Day", "S_D_45_Fructose_c[C_c]_Day", "S__40_S_41__45_Malate_c[C_c]_Day", "S_Fumarate_c[C_c]_Day", "S_Citrate_c[C_c]_Day"]),
    PhotonRestrictor(modelo, "Ex16_Night"),
    Biomass(modelo, "BIO_L_Day", "BIO_L_Night", ["R03845_c_Night", "R06267_c_Night", "R06286_c_Night", "R04801_c_Night", "R07856_c_Night", "R04803_c_Night", "R06961_c_Night", "R06960_c_Night", "R07856_c_Night", "V0004_Night", "R06284_c_Night", "R05618_c_Night", "R03845_c_Night", "R03824_c_Night", "R05341_c_Night", "R06965_c_Night", "R07558_c_Night", "R04798_c_Night", "R04787_c_Night", "R07840_c_Night", "R06963_c_Night", "R06962_c_Night", "R06961_c_Night"])
]

pipeline = Pipeline(modelo, Steps)
pipeline.run()

In [3]:
res_model = pipeline.model
res_model

Name,M_
Memory address,2dc2c375fa0
Number of metabolites,3474
Number of reactions,3202
Number of genes,1404
Number of groups,306
Objective expression,0.01*Ex16_Day - 0.01*Ex16_Day_reverse_efba9
Compartments,"unknownCompartment3 Day, unknownCompartment6 Day, unknownCompartment5 Day, unknownCompartment8 Day, unknownCompartment7 Day, unknownCompartment2 Day, unknownCompartment1 Day, unknownCompartment4 Day, unknownCompartment3 Night, unknownCompartment2 Night, unknownCompartment6 Night, unknownCompartment1 Night, unknownCompartment4 Night, unknownCompartment5 Night, unknownCompartment7 Night, unknownCompartment8 Night"
